In [1]:
1

1

create dict from summary db  
id1n:  ym: total cost in the ym

In [2]:
using Distributed
thrdnum = 4
addprocs(thrdnum)

4-element Vector{Int64}:
 2
 3
 4
 5

In [3]:
@everywhere using Mongoc
@everywhere using Dates

In [4]:
@everywhere using JSON
@everywhere using JLD2
@everywhere using HDF5

# using Profile

In [5]:
@everywhere using Dates
@everywhere using Tables
@everywhere using DataFrames
@everywhere using CSV
@everywhere using XLSX
@everywhere using BSON
@everywhere using Distributed
@everywhere using FileIO
# using Printf
# using Query

# addprocs(8)

In [6]:
@everywhere using DataFramesMeta

In [7]:
# d_iy
# @everywhere using Mongoc
# @everywhere using Dates
# @everywhere using DataFrames
# @everywhere using DataFramesMeta
# @everywhere using JLD2
# @everywhere using CSV

In [8]:
@everywhere function gen_dict()
    f_mst = "../dicts/Jm01_dis_parse/byomei_cancer_20210701.csv"
    df_mst = @chain CSV.read(f_mst, header=0,types=Dict(4=>String), DataFrame) begin
            @transform(:tag = first.(:Column17, 3))
        end
    d_dis2code2 = Dict{String, String}()
    for r in eachrow(df_mst)
        d_dis2code2[r["Column4"]] = r["tag"]
        # print(r["Column4"])
        # break
    end


        
    d_cs = Dict{Vector{String}, Dict{String, Dict}}()
    d_ids = Dict{String, Dict{Int32, Any}}()
    
    d_ids_each = Dict{String, Any}()
    d_ids_each["first_ym"] = 0
    d_ids_each["gender"] = 0
    d_ids_each["first_cancers"] = []
    d_ids_each["ym_count"] = 0
    d_ids_each["last_ym"] = 0

    d_ym = Dict{Int32, Float32}()
    d_ymd = Dict{Int32, Float32}()
    for y in 2011:2022
        # println(string(y))
        for m in 1:12
            ym = string(y) * lpad(m, 2, "0")
            d_ym[parse(Int32, ym)] = 0.0
            # println(lpad(m, 2, "0"))
            for d in 1:31
                ymd = ym * lpad(d, 2, "0")
                d_ymd[parse(Int32, ymd)] = 0.0
                # println(lpad(d, 2, "0"))
            end
        end
    end

    f_iy = "../dicts/Jm01_dis_parse/dfmeta_iy_20231122_150053_929635.csv"
    df_iy = @chain CSV.read(f_iy, header=1,types=Dict("c2"=>String), DataFrame) begin
            end
    d_iy = Dict{String, Dict{Int32, Float32}}()
    for r in eachrow(df_iy)
        d_iy[r["c2"]] = copy(d_ymd)
    end
    
    
    println("base dicts generated")
    return d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy
end

In [9]:
# const c_ids_each = d_ids_each
# const c_dis2code2 = d_dis2code2 
# const c_ym = d_ym
# const c_ymd = d_ymd
# const c_iy = d_iy

In [10]:
dbs = ["s_" * lpad(i, 3, '0') for i in 0:255]
const dbs2d = reshape(dbs, thrdnum, :)

4×64 Matrix{String}:
 "s_000"  "s_004"  "s_008"  "s_012"  …  "s_240"  "s_244"  "s_248"  "s_252"
 "s_001"  "s_005"  "s_009"  "s_013"     "s_241"  "s_245"  "s_249"  "s_253"
 "s_002"  "s_006"  "s_010"  "s_014"     "s_242"  "s_246"  "s_250"  "s_254"
 "s_003"  "s_007"  "s_011"  "s_015"     "s_243"  "s_247"  "s_251"  "s_255"

In [11]:
# const dbs = ["s_" * lpad(i, 3, '0') for i in 0:255]

In [12]:
# ../../../storage/smr_tabl/smr_tbl_id1ns_s_231.jld2
@everywhere filelist = readdir("../../../storage/smr_tabl/")
# @everywhere 
@everywhere filelist = [f for f in filelist if occursin("smr_tbl_id1ns_s", f)]
println(filelist[1:3])
length(filelist)

["smr_tbl_id1ns_s_000.jld2", "smr_tbl_id1ns_s_001.jld2", "smr_tbl_id1ns_s_002.jld2"]


256

In [13]:
a = Dict{Any, Any}()    
a[1] = 0

0

In [14]:
@everywhere function e2d_add(d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy, e)
    #not consider cancer
    id1n = e["id1n"]       
    ym = e["ym"]
    # ym =parse(Int32, ym_s)
    

    if !haskey(d_ids, id1n)
        d_ids[id1n] = Dict{Int32, Float64}()            
    end

    if !haskey(d_ids[id1n], ym)
        d_ids[id1n][ym] = 0
    end
    d_ids[id1n][ym] += e["totalpoint"]
        
        # for (dc,dv) in e["usage"]["iy"]
        #     if !haskey(d_cs[cs]["iy"], dc)
        #         d_cs[cs]["iy"][dc] = copy(d_ymd)
        #     end
            
        #     for (k, v) in dv
        #         d_cs[cs]["iy"][dc][parse(Int32, ym_s * k)] += v
        #     end
        # end
        

    # end
end

In [15]:
# dbs

In [16]:
@everywhere function create_dicts_cost_sssub(ss)
    
    a = now()
    client = Mongoc.Client("192.168.0.25", 27017)
    db = client["smr"]
    
    for s in ss
        d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy = gen_dict()
        println(s)
        collection = db[s]
        
        a2 = now()
        for (i,e) in enumerate(Mongoc.find(collection))
            e2d_add(d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy,e)
            if i % 1e+6 == 0
                # break
                print(", checkpoint:")
                print(i)
                b = now()
            
                print(" time:")
                print(canonicalize(b -a))
                print(" chunktime:")
                print(canonicalize(b -a2))
                # print(" dictsize:")
                # print(length(d_id1n))
                print("\n")
                a2 = now()
            end
        
        end
        f = get_path_cost(s)
        println(f)
        save(f,"data", d_ids) 
        d_ids = nothing
    end
    
    print(":loop finished:")
    # f = get_path_cs("whole")
    println(f)
    # save(f,"data", d_cs)
    # save(get_path_id1s(s),"data", d_ids)
    return 1
end

In [17]:
# get_path(x1) = "../../../storage/smr_tabl/smr_tbl_$x1.jld2"
# @everywhere  
@everywhere get_path_cost(x1) = "../../../storage/smr_tabl/smr_tbl_cost_$x1.jld2"
# @everywhere  
# get_path_id1s(x1) = "../../../storage/smr_tabl/smr_tbl_id1ns_$x1.jld2"
get_path_cost("aaa")
# get_path_id1s("aaa")

"../../../storage/smr_tabl/smr_tbl_cost_aaa.jld2"

In [18]:
dbs2d

4×64 Matrix{String}:
 "s_000"  "s_004"  "s_008"  "s_012"  …  "s_240"  "s_244"  "s_248"  "s_252"
 "s_001"  "s_005"  "s_009"  "s_013"     "s_241"  "s_245"  "s_249"  "s_253"
 "s_002"  "s_006"  "s_010"  "s_014"     "s_242"  "s_246"  "s_250"  "s_254"
 "s_003"  "s_007"  "s_011"  "s_015"     "s_243"  "s_247"  "s_251"  "s_255"

In [19]:
# for i=1:size(dbs2d, 1)
#     ss = dbs2d[i,:]
#     println(ss)
# end
# create_dicts_cost_sssub(dbs2d[1,:])

In [20]:
results = @distributed (+) for i=1:size(dbs2d, 1)
    ss = dbs2d[i,:]
    create_dicts_cost_sssub(ss)
    
    # println(dbxs)
end

      From worker 3:	base dicts generated
      From worker 3:	s_001
      From worker 5:	base dicts generated
      From worker 5:	s_003
      From worker 4:	base dicts generated
      From worker 4:	s_002
      From worker 2:	base dicts generated
      From worker 2:	s_000
      From worker 5:	, checkpoint:1000000 time:24 seconds, 474 milliseconds chunktime:5 seconds, 974 milliseconds
      From worker 2:	, checkpoint:1000000 time:24 seconds, 509 milliseconds chunktime:5 seconds, 992 milliseconds
      From worker 3:	, checkpoint:1000000 time:24 seconds, 908 milliseconds chunktime:6 seconds, 433 milliseconds
      From worker 4:	, checkpoint:1000000 time:25 seconds, 825 milliseconds chunktime:7 seconds, 368 milliseconds
      From worker 5:	, checkpoint:2000000 time:32 seconds, 991 milliseconds chunktime:8 seconds, 400 milliseconds
      From worker 2:	, checkpoint:2000000 time:32 seconds, 911 milliseconds chunktime:8 seconds, 258 milliseconds
      From worker 3:	, checkpoint:200000

LoadError: TaskFailedException

[91m    nested task error: [39mOn worker 2:
    UndefVarError: f not defined
    Stacktrace:
     [1] [0m[1mcreate_dicts_cost_sssub[22m
    [90m   @ [39m[90m./[39m[90m[4mIn[16]:40[24m[39m
     [2] [0m[1mmacro expansion[22m
    [90m   @ [39m[90m./[39m[90m[4mIn[20]:3[24m[39m[90m [inlined][39m
     [3] [0m[1m#7[22m
    [90m   @ [39m[90m/opt/julia-1.8.5/share/julia/stdlib/v1.8/Distributed/src/[39m[90m[4mmacros.jl:288[24m[39m
     [4] [0m[1m#invokelatest#2[22m
    [90m   @ [39m[90m./[39m[90m[4messentials.jl:729[24m[39m
     [5] [0m[1minvokelatest[22m
    [90m   @ [39m[90m./[39m[90m[4messentials.jl:726[24m[39m
     [6] [0m[1m#110[22m
    [90m   @ [39m[90m/opt/julia-1.8.5/share/julia/stdlib/v1.8/Distributed/src/[39m[90m[4mprocess_messages.jl:285[24m[39m
     [7] [0m[1mrun_work_thunk[22m
    [90m   @ [39m[90m/opt/julia-1.8.5/share/julia/stdlib/v1.8/Distributed/src/[39m[90m[4mprocess_m

In [1]:
# results

In [ ]:
# create_dicts_cost(dbs)